# TL;DR Solution:

Problem states: You are given a training set with roughly **~2%** of the data labeled.

Since manual labelling and pretrained models (except resnet18) are prohibited, only way to achieve good score is self-supervised learning (SSL)

SSL is a method, when we use the unlabeled data for models training. Usage of unlabaled data varies over SSL methods. To understand this method better, lets overview some examples of SSL. Popular and most useful SSL:

1. Self training with noisy student: https://arxiv.org/pdf/1911.04252
You first train the teacher model on labeled data, and generate pseudo labels for the unlabeled data. Then you create new student model (Which is equal or larger than teacher), which is trained on pseudo labels and labeled data too. For better generalization, we inject noise in student model and data itself (for model we use dropout and stoch depth, and for data we use augmentations), which improves the performance of student model and reduces variance.

2. FixMatch: https://arxiv.org/pdf/2001.07685
We train the model on labeled data. For inference, we use weak augmentation to predict class probability and we use confidence threshold (Ex: predicting class probability). If the probability is higher than conf threshold, we chose it as pseudo label for class. After this, we use stronger augmentations on all the images and train the model another time. Training objective is to match pseudo labels with the strong-augmented training models prediction.

For the solution, we used fixmatch and achieved 0.79 on private leaderboard (Noisy student might take too much time).



In [ ]:
import os
from pathlib import Path

import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2 as T
from torchvision.models import ResNet18_Weights, resnet18

I was writing code in colab so i unzipped train/test folders

In [ ]:
!unzip train.zip

Archive:  train.zip
   creating: train/
  inflating: train/0000.jpg          
  inflating: train/0001.jpg          
  inflating: train/0002.jpg          
  inflating: train/0003.jpg          
  inflating: train/0004.jpg          
  inflating: train/0005.jpg          
  inflating: train/0006.jpg          
  inflating: train/0007.jpg          
  inflating: train/0008.jpg          
  inflating: train/0009.jpg          
  inflating: train/0010.jpg          
  inflating: train/0011.jpg          
  inflating: train/0012.jpg          
  inflating: train/0013.jpg          
  inflating: train/0014.jpg          
  inflating: train/0015.jpg          
  inflating: train/0016.jpg          
  inflating: train/0017.jpg          
  inflating: train/0018.jpg          
  inflating: train/0019.jpg          
  inflating: train/0020.jpg          
  inflating: train/0021.jpg          
  inflating: train/0022.jpg          
  inflating: train/0023.jpg          
  inflating: train/0024.jpg          
  inflatin

In [ ]:
!unzip test.zip

Archive:  test.zip
   creating: test/
  inflating: test/0000.jpg           
  inflating: test/0001.jpg           
  inflating: test/0002.jpg           
  inflating: test/0003.jpg           
  inflating: test/0004.jpg           
  inflating: test/0005.jpg           
  inflating: test/0006.jpg           
  inflating: test/0007.jpg           
  inflating: test/0008.jpg           
  inflating: test/0009.jpg           
  inflating: test/0010.jpg           
  inflating: test/0011.jpg           
  inflating: test/0012.jpg           
  inflating: test/0013.jpg           
  inflating: test/0014.jpg           
  inflating: test/0015.jpg           
  inflating: test/0016.jpg           
  inflating: test/0017.jpg           
  inflating: test/0018.jpg           
  inflating: test/0019.jpg           
  inflating: test/0020.jpg           
  inflating: test/0021.jpg           
  inflating: test/0022.jpg           
  inflating: test/0023.jpg           
  inflating: test/0024.jpg           
  inflating:

In [ ]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

batch_size = 16

device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, is_train: bool):
        super().__init__()

        self.is_train = is_train
        self.ds_path = f"{'train' if is_train else 'test'}"

        self.paths = os.listdir(self.ds_path)

        self.transforms = T.Compose([
            T.Resize((224, 224)),
            T.PILToTensor(),
            T.ToDtype(torch.float32, scale=True),
        ])

    def __getitem__(self, idx):
        # format: ./dataset/test/0040.jpg
        img_path = f"{self.ds_path}/{self.paths[idx]}"
        img = Image.open(img_path).convert("RGB")

        img = self.transforms(img)
        return img

    def __len__(self):
        return len(self.paths)

In [ ]:
test_ds = BrainTumorDataset(is_train=False)

test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [ ]:
resnet = resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
resnet.fc = nn.Identity()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 90.3MB/s]


In [ ]:
# HP

CONFIDENCE = 0.95
UNLABELED_WEIGHT = 1.0 # FixMatch Loss = L1 (Labeled) + lambda*L2(Unlabeled)
LEARNING_RATE = 3e-4
EPOCHS = 22
BATCH_SIZE = 16
UNLABELED_BATCH_RATIO = 4

In [ ]:
TRAIN_DIR = "train"
TEST_DIR = "test"
TRAIN_CSV_PATH = "train.csv"

In [ ]:
labeled_df = pd.read_csv(TRAIN_CSV_PATH)

labeled_df['image_path'] = labeled_df['image_id'].apply(lambda x: f"{TRAIN_DIR}/{x:04d}.jpg")

In [ ]:
all_train_files = os.listdir(TRAIN_DIR)

labeled_files = set(labeled_df['image_id'].apply(lambda x: f"{x:04d}.jpg"))
unlabeled_files = [f for f in all_train_files if f not in labeled_files] # every file in train except labeled ones

unlabeled_df = pd.DataFrame({'image_path': [os.path.join(TRAIN_DIR, f) for f in unlabeled_files]})

In [ ]:
labels = sorted(labeled_df['label'].unique())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for i, lbl in enumerate(labels)}
labeled_df['label_id'] = labeled_df['label'].map(label2id)

In [ ]:
print(f"Labeled samples: {len(labeled_df)}")
print(f"Unlabeled samples: {len(unlabeled_df)}")
print(f"Total train samples: {len(labeled_df) + len(unlabeled_df)}")
print(f"Label mapping: {label2id}")

Labeled samples: 48
Unlabeled samples: 1952
Total train samples: 2000
Label mapping: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


As we've said, for fixmatch, the strong and weak augmentations are needed.

In [ ]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
weak_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    normalize,
])
strong_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandAugment(num_ops=2, magnitude=10),
    T.ToTensor(),
    normalize,
    T.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0),
])
test_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    normalize,
])

/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
class LabeledBrainTumorDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image_path']).convert("RGB")
        img = self.transform(img)
        label = torch.tensor(row['label_id'], dtype=torch.long)
        return img, label

class UnlabeledBrainTumorDataset(Dataset):
    def __init__(self, df, weak_transform, strong_transform):
        self.df = df
        self.weak_transform = weak_transform
        self.strong_transform = strong_transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image_path']).convert("RGB")
        img_weak = self.weak_transform(img)
        img_strong = self.strong_transform(img)
        return img_weak, img_strong

class TestDataset(Dataset):
    def __init__(self, test_dir, transform):
        self.paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir)]
        self.transform = transform
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        return img, os.path.basename(img_path)

In [ ]:
labeled_ds = LabeledBrainTumorDataset(labeled_df, weak_transform) # weak transforms for labeled
unlabeled_ds = UnlabeledBrainTumorDataset(unlabeled_df, weak_transform, strong_transform) # strong transforms for the second phase of training
test_ds = TestDataset(TEST_DIR, test_transform)

labeled_loader = DataLoader(labeled_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
unlabeled_loader = DataLoader(unlabeled_ds, batch_size=BATCH_SIZE * UNLABELED_BATCH_RATIO, shuffle=True, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=2)

In [ ]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 4) # output is 4 because we have 4 labels.
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
sup_loss_fn = nn.CrossEntropyLoss() # CE is classic loss for fixmatch
unsup_loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
for epoch in range(EPOCHS): # Training
    model.train()
    total_loss = 0
    labeled_iter = iter(labeled_loader)
    progress_bar = tqdm(unlabeled_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for (unlabeled_weak, unlabeled_strong) in progress_bar:
        try:
            labeled_imgs, labels = next(labeled_iter)
        except StopIteration:
            labeled_iter = iter(labeled_loader)
            labeled_imgs, labels = next(labeled_iter)

        labeled_imgs, labels = labeled_imgs.to(device), labels.to(device)
        unlabeled_weak, unlabeled_strong = unlabeled_weak.to(device), unlabeled_strong.to(device)

        sup_preds = model(labeled_imgs)
        loss_sup = sup_loss_fn(sup_preds, labels)

        with torch.no_grad():
            pseudo_preds = model(unlabeled_weak)
            pseudo_probs = torch.softmax(pseudo_preds, dim=1) # logits -> probability
            max_probs, pseudo_labels = torch.max(pseudo_probs, dim=1)
            mask = max_probs.ge(CONFIDENCE).float() # pred >= confidence threshold

        strong_preds = model(unlabeled_strong)
        loss_unsup_per_sample = unsup_loss_fn(strong_preds, pseudo_labels)
        loss_unsup = (loss_unsup_per_sample * mask).mean()

        loss = loss_sup + UNLABELED_WEIGHT * loss_unsup # loss = L1 + lambda*L2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item(), sup_loss=loss_sup.item(), unsup_loss=loss_unsup.item(), mask_ratio=mask.mean().item())

    scheduler.step()
    print(f"Epoch {epoch+1} Average Loss: {total_loss / len(unlabeled_loader):.4f}")

Epoch 1/22: 100%|██████████| 31/31 [00:23<00:00,  1.32it/s, loss=0.108, mask_ratio=0.812, sup_loss=0.00242, unsup_loss=0.106]

Epoch 1 Average Loss: 0.2272



Epoch 2/22: 100%|██████████| 31/31 [00:21<00:00,  1.45it/s, loss=0.203, mask_ratio=0.844, sup_loss=0.000789, unsup_loss=0.202]

Epoch 2 Average Loss: 0.1315



Epoch 3/22: 100%|██████████| 31/31 [00:21<00:00,  1.42it/s, loss=0.0144, mask_ratio=0.844, sup_loss=0.00651, unsup_loss=0.00786]

...

Epoch 20 Average Loss: 0.0363



Epoch 21/22: 100%|██████████| 31/31 [00:21<00:00,  1.42it/s, loss=0.0664, mask_ratio=0.938, sup_loss=7.7e-5, unsup_loss=0.0663]

Epoch 21 Average Loss: 0.0304



Epoch 22/22: 100%|██████████| 31/31 [00:23<00:00,  1.35it/s, loss=0.00591, mask_ratio=0.969, sup_loss=8.95e-5, unsup_loss=0.00582]

Epoch 22 Average Loss: 0.0271


Saving submission

In [ ]:
model.eval()
all_preds = []
all_ids = []
with torch.no_grad():
    for imgs, fnames in tqdm(test_loader, desc="Predicting"):
        imgs = imgs.to(device)
        preds_logits = model(imgs)
        preds_indices = torch.argmax(preds_logits, dim=1).cpu().numpy()
        all_preds.extend(preds_indices)
        all_ids.extend(fnames)

preds_labels = [id2label[p] for p in all_preds]
ids = [fname.replace(".jpg", "") for fname in all_ids]

sub = pd.DataFrame({"ID": ids, "prediction": preds_labels})

sub = sub.sort_values(by="ID").reset_index(drop=True)
sub.to_csv("submission.csv", index=False)

sub.head()

Predicting: 100%|██████████| 41/41 [00:06<00:00,  6.55it/s]


,ID,prediction
0,0000,notumor
1,0001,meningioma
2,0002,notumor
3,0003,notumor
4,0004,glioma


We didnt break any rules of problem:

* Runtime: 10min on T4/P100 16gb VRAM

* Didnt label manually

* Didnt use any other models or weights except resnet18